In [1]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
train = pd.read_csv('input/train.csv')

train['Gender'] = train['Gender'].map({'Female':1, 'Male':0})
train = train.drop('id', axis=1)

# AutoML

In [3]:
train_tab = TabularDataset(train)

predictor = (
    TabularPredictor(
        label='Exited', 
        problem_type='binary',
        eval_metric='roc_auc')
    .fit(train_tab, presets='best_quality')
)

No path specified. Models will be saved in: "AutogluonModels\ag-20240105_160544"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240105_160544/ds_sub_fit/sub_fit_ho.
2024-01-05 13:05:45,774	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning Au

# Results

In [32]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.896839,roc_auc,91.211097,2395.773344,0.044402,60.079626,3,True,16
1,CatBoost_BAG_L2,0.896525,roc_auc,51.846303,2032.596923,0.529573,414.795512,2,True,13
2,WeightedEnsemble_L2,0.896012,roc_auc,29.036007,1620.113429,0.027661,33.135044,2,True,10
3,CatBoost_BAG_L1,0.895469,roc_auc,1.276476,1091.897413,1.276476,1091.897413,1,True,5
4,ExtraTreesGini_BAG_L2,0.894563,roc_auc,62.315932,1634.871566,10.999203,17.070154,2,True,14
5,ExtraTreesEntr_BAG_L2,0.894421,roc_auc,62.802607,1637.633580,11.485877,19.832168,2,True,15
6,RandomForestEntr_BAG_L2,0.892404,roc_auc,59.138185,1758.269853,7.821456,140.468442,2,True,12
7,RandomForestGini_BAG_L2,0.891444,roc_auc,60.330585,1743.527442,9.013856,125.726031,2,True,11
8,XGBoost_BAG_L1,0.891187,roc_auc,3.652054,58.935897,3.652054,58.935897,1,True,9
9,NeuralNetFastAI_BAG_L1,0.888268,roc_auc,5.583591,326.413980,5.583591,326.413980,1,True,8


# Submission

In [23]:
test_df = pd.read_csv('input/test.csv')
test_df['Gender'] = test_df['Gender'].map({'Female':1, 'Male':0})
test_data = TabularDataset(test_df)
y_pred = predictor.predict_proba(test_data)[1]

sub_sample = pd.read_csv('input/sample_submission.csv')
sub_sample['Exited'] = y_pred

In [31]:
sub_sample.to_csv('submission1.csv', index=False)